# Basic Multi-Agent Example with AutoGen

This notebook implements a simple multi-agent system using AutoGen. 

In [ ]:
import os, nest_asyncio
nest_asyncio.apply()
import asyncio
import timeit
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.tools import TeamTool
import sys

sys.path.append(os.path.abspath(os.path.join(os.path.curdir, "..")))

from tools import (
    create_chat_completion_client,
    current_datetime_utc_tool,
    adjust_properties_for_fixed_response,
    get_tavily_search_tool,
    get_attributes,
    load_model_config,
)

#### Note: You need a tavily API key to use the web search tool!

In [43]:
model = load_model_config()

Using model: gpt-4.1-nano type: openai


Set the joke's topic
----------------------

In [44]:
topic = "Animals"

Select Model and Agent Configuration
-----------------------------

In [45]:
agentConfig = {
    "Author": dict(
        system_message="Write a SHORT joke above the provided topic."
    ),
    "SmartassEditor": dict(
        system_message="Analyze the joke, explain why it is funny or not funny, and make it funnier without making it longer.",
    ),
}

# For testing, we can fix an agent's response to a known answer
#adjust_properties_for_fixed_response(agentConfig["Author"], "What do you call a magic dog? A Labracadabrador.")

In [46]:
# Now create model client
model_client = create_chat_completion_client(model)

agents = []  # start with a user proxy for the researcher

for agent_type, props in agentConfig.items():
    agents.append(
        AssistantAgent(
            name=agent_type,
            **props,
            model_client=model_client,
        )
    )

In [47]:
# Set up termination conditions
termination = MaxMessageTermination(max_messages=10) #len(agents))

# Create the team
team = RoundRobinGroupChat(
    participants=agents,
    termination_condition=termination,
    max_turns=len(agents) # we don't really want it to go round
)


Start the System
===================

In [48]:
async def invent_joke(topic: str):
    start_time_s = timeit.default_timer()
    result = await team.run(task=f"Invent a short joke about: {topic}")
    elapsed = timeit.default_timer()-start_time_s
    print(f"🔍 Generation result (took {elapsed:.2f} sec.) resulted in {len(result.messages)} messages.")
    return result  # Return the result for further exploration

# Run the deep dive
result = await invent_joke(topic)

🔍 Generation result (took 1.14 sec.) resulted in 3 messages.


Explore Results
===============

In [49]:
# Display the full result structure
print("Result type:", type(result))

# print("\nResult attributes:")
# print(get_attributes(result))
# print("\nMessage attributes:")
# print(get_attributes(result.messages[-1]))

print("Message Sequence:")
for i, m in enumerate(result.messages):
    print(f"\n----------------------------------------------------------------")
    print(f"MESSAGE {i} [{m.source}]:\n{m.content}")
    

Result type: <class 'autogen_agentchat.base._task.TaskResult'>
Message Sequence:

----------------------------------------------------------------
MESSAGE 0 [user]:
Invent a short joke about: Animals

----------------------------------------------------------------
MESSAGE 1 [Author]:
Why did the squirrel bring a ladder to the tree?  
Because it wanted to get to the nutty level!

----------------------------------------------------------------
MESSAGE 2 [SmartassEditor]:
The joke is funny because it plays on the double meaning of "nutty," referring both to the squirrel’s love for nuts and a humorous "higher level" — like climbing a ladder to reach the next "stage" of nuttiness. It’s a clever pun that combines animal behavior with a common phrase.

To make it funnier without adding length:  
"Why did the squirrel bring a ladder? To reach the top nut-venture!"


In [50]:
# Clean up resources
await model_client.close()

In [51]:
import re
from autogen_agentchat.messages import BaseTextChatMessage

print("Stop reason:", result.stop_reason)
assert isinstance(result.messages[-1], BaseTextChatMessage)

print("Message Sequence:")
for m in result.messages:
    assert isinstance(m, BaseTextChatMessage), f"Unexpected message type: {type(m)}"
    message = m.content
    print(f" - {m.source}: {message}")


Stop reason: Maximum number of turns 2 reached.
Message Sequence:
 - user: Invent a short joke about: Animals
 - Author: Why did the squirrel bring a ladder to the tree?  
Because it wanted to get to the nutty level!
 - SmartassEditor: The joke is funny because it plays on the double meaning of "nutty," referring both to the squirrel’s love for nuts and a humorous "higher level" — like climbing a ladder to reach the next "stage" of nuttiness. It’s a clever pun that combines animal behavior with a common phrase.

To make it funnier without adding length:  
"Why did the squirrel bring a ladder? To reach the top nut-venture!"
